In [1]:
# Install a pip package in the current Jupyter kernel
#import sys
#!{sys.executable} -m pip install faker
#!{sys.executable} -m pip install geopy
#!{sys.executable} -m pip install arcgis
#!{sys.executable} -m pip install folium

In [2]:
import pandas as pd
from random import randint
import random
import numpy as np
import calendar

from faker import Faker
from faker.providers import internet, geo
from random import gauss

from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

In [3]:
medicines_ = pd.read_excel('../data/raw/LM_noviembre_2018.xlsx')

In [4]:
import re
def lowerNameColumns(df):
    columns = [re.sub('\s+', '_', x.lower()) for x in df.columns.tolist()]
    return columns

# Column names normalization
medicines_.columns = lowerNameColumns(medicines_)


In [5]:
medicines = medicines_.copy()

# Do some cleaning
medicines['nivel'].fillna('X', inplace=True)

index = medicines[medicines['nivel'].str.contains(r'[\d].[\d].[\d]')].index.values
medicines.loc[index, 'nivel'] = 'X'

index = medicines[medicines['nivel'].str.contains(r'[\d].[\d]')].index.values
medicines['subcategoria'] = medicines.loc[index,'nivel']
medicines.loc[index, 'nivel'] = 'X'

index = medicines[medicines['nivel'].str.contains(r'[\d].')].index.values
medicines['categoria'] = medicines.loc[index,'nivel']
medicines.loc[index, 'nivel'] = 'X'

medicines['subcategoria'].fillna(method='ffill', inplace=True)
medicines['categoria'].fillna(method='ffill', inplace=True)

index = medicines[medicines['nivel'].str.lower()==medicines.columns[0]].index.values
medicines.loc[index, 'nivel'] = 'X'

medicines = medicines[medicines['nivel']!='X'].reset_index(drop=True)

medicines.rename(columns={'forma,_concentración_y_presentación':'presentación',
                         'nombre_genérico_(dci)':'nombre_genérico'}, inplace=True)

medicines['código'] = medicines['código'].astype('int')

medicines['subcategoria'] = medicines['subcategoria'].replace('\d\S+\s','', regex=True)
medicines['categoria'] = medicines['categoria'].replace('\d\S+\s','', regex=True)

In [6]:
# Select only category of medicine to construct the pivot
category = 'Antihipertensivos'
antihiper_codes = medicines[medicines['subcategoria']==category]['código'].to_list()
antihiper_subcat = medicines[medicines['subcategoria']==category]['nombre_genérico'].to_list()
print('nro. de antihipertensivos :', len(antihiper_codes))

# Generate random products, items and prices
N = 1000
choice = np.random.choice([0,1], p=[0.7,0.3], size=(N, len(antihiper_codes)))
quantity_ = np.random.choice([1,2,3,4], p=[.5, .3, .15, .05], size=(N, len(antihiper_codes)))
quantity = choice * quantity_

prices = np.array([round(random.uniform(10.5, 50.5),2) for x in range(len(antihiper_codes))])
totalprice = prices * quantity

# convert pivot into dataframe with unstack
pivot_q = pd.DataFrame(data=quantity, columns=antihiper_codes)
pivot_q.replace(0,np.NaN, inplace=True)
pivot_p = pd.DataFrame(data=totalprice, columns=antihiper_codes)
pivot_p.replace(0,np.NaN, inplace=True)

dfq = pivot_q.unstack(level=0).reset_index()
dfq.rename(columns={'level_0':'code_m', 'level_1':'cliente_id', 0:'cantidad'}, inplace=True)
dfq.dropna(subset=['cantidad'], inplace=True)
dfp = pivot_p.unstack(level=0).reset_index()
dfp.rename(columns={'level_0':'code_m', 'level_1':'cliente_id', 0:'precio_total'}, inplace=True)
dfp.dropna(subset=['precio_total'], inplace=True)
df = pd.concat([dfq, dfp[['precio_total']]], axis=1)
df = df[['cliente_id', 'code_m', 'cantidad', 'precio_total']].reset_index(drop=True)

nro. de antihipertensivos : 29


In [7]:
df[df['cliente_id']==500]

,cliente_id,code_m,cantidad,precio_total
4402,500,2059,4.0,78.88
5026,500,624,2.0,74.84
5654,500,2003,2.0,100.22


In [8]:
# make fake data
fake = Faker()
ROWS = 10000

In [9]:
products = {
    # Product : [Price, weight]  # weight to do it more choiceable
    'Apple iPhone11 Smartphone':[3379, 5],
    'Samsung GalaxyS10 Smartphone':[2329, 5],
    'Motorola MotoG8 Smartphone':[789, 25],
    'Huawei Y6 Smartphone':[1149, 15],
    'Apple MacBookPro Laptop':[11099, 2],
    'Lenovo Ideapad Laptop':[2499, 5],
    'Asus Ryzen Laptop':[2199, 5],
    'Acer SF314-41 Laptop':[2799, 5],
    'Skullcandy Headphones':[419, 30],
    'Apple Airpods(2da generación) Headphones':[629, 25],
    'Beats Studio3 Headphones':[1499, 15],
    'Bose SoundSport Headphones':[549, 25],
    'Bose Soundlink Speaker':[899, 25],
    'JBL Go2 Speaker':[129, 30],
    'Ultimate Ears Speaker':[329, 30],
    'Sonos Play5 Speaker':[2499, 5],
    'Apple Series5 Watch':[2499, 5],
    'Garmin Forerunner645 Watch':[1919.9, 15],
    'Radioshack Smart Watch':[189.9, 30],
    'Huawei GT2E Watch':[549, 25],
    'Mophie Powerstation Battery':[349, 30],
    'Belkin USB-C Charger':[179, 30],
    'Palace Universal Case':[129, 30],
    'NCO Universal GlassGuard':[99, 30]
}

In [10]:
from datetime import timedelta
from datetime import datetime

def generate_random_time(month):
    # Generate a random date with two peaks at 12 and 20
    YEAR = 2020
    day_range = calendar.monthrange(YEAR, month)[1]
    random_day = random.randint(1, day_range)
    
    if random.random() > 0.5:
        date = datetime(YEAR, month, random_day, 12,0)
    else:
        date = datetime(YEAR, month, random_day, 20,0)
        
    time_offset = np.random.normal(loc=0, scale=180)
    final_date = date + timedelta(minutes=time_offset)
    
    return final_date
    
    
def generate_random_address():
    # Generate random address
    addresses = ['Av. Javier Prado Oeste 770, Magdalena, Magdalena, Lima', 
                 'Av. Cuba 1145, Jesús María, Jesús María, Lima',
                 'Calle Montemar 167, Santiago de Surco, Santiago de Surco, Lima', 
                 'Av. Brasil 1670 , Pueblo Libre, Lima', 
                 'Nicolás de Piérola 140, San Miguel, Lima', 
                 'Alfredo Salazar 154, Miraflores, Lima', 
                 'Micaela Bastidas 1100, El Retablo, Comas',
                 'los cisnes 371, Limatambo, San Isidro',
                 'Av. Cesar Vallejo 223, Lince, Lince, Lima',
                 'Av. Chorrillos 640, Chorrillos, Lima',
                 'Av. 10 de Junio 1020, San Martín de Porres, Lima', 
                 'Av. Angamos Este 1551, Surquillo, Lima', 
                 'Granada 135, Pueblo Libre, Lima',
                 'Av. Petit Thouars 1291, Santa Beatriz, Lima']
    
    weights = [9,4,5,2,3,3,2,0.5,6,3,5,1,7,8]
    
    izibussiness = [8294148, 8294149, 8294150, 8294151, 8294152, 8294153, 8294154, 
                    8294155, 8294156, 8294157, 8294158, 8294159, 8294160, 8294161]
    
    index = random.choices(range(len(addresses)), weights=weights)[0]
    
    return addresses[index], izibussiness[index]

def write_row(order_id, product, date, address):
    price = products[product][0]
    quantity = np.random.geometric(p=1-(10/price), size=1)[0]
    
    return [order_id, product, quantity, price, date, address]
    

In [11]:

cards = [fake.credit_card_number() for x in range(ROWS)]

columns = ['orderid', 'product', 'quantity', 'price', 'orderdate', 
           'address']

product_list = [product for product in products]
weights = [products[product][1] for product in products]
order_id = 14356

for month_value in range(1, 13):
    if month_value<=10:
        #orders_amount = int(np.random.normal(loc=12000, scale=4000))
        orders_amount = 100
    elif month_value==11:
         orders_amount = int(np.random.normal(loc=20000, scale=3000))
    elif month_value==12:
         orders_amount = int(np.random.normal(loc=26000, scale=3000))
            
    df = pd.DataFrame(columns=columns)
    
    i = 0

    while orders_amount > 0:
        address, izibussiness = generate_random_address()
        date = generate_random_time(month_value)
        product = random.choices(product_list, weights=weights)[0]
        df.loc[i] = write_row(order_id, product, date, address)
        i += 1
        
        if product=='Apple iPhone11 Smartphone':
            if random.random() < 0.15:
                df.loc[i] = write_row(order_id, 
                                      'Apple Airpods(2da generación) Headphones', 
                                      date, address)
                i += 1
            if random.random() < 0.07:
                df.loc[i] = write_row(order_id, 'JBL Go2 Speaker', 
                                      date, address)
                i += 1
        elif product=='Samsung GalaxyS10 Smartphone' or product=='Huawei Y6 Smartphone':
            if random.random() < 0.18:
                df.loc[i] = write_row(order_id, 
                                        'NCO Universal GlassGuard', 
                                         date, address)
                i += 1
        
        if random.random() <= 0.02:
            product2 = random.choices(product_list, 
                                      weights=weights)[0]
            df.loc[i] = write_row(order_id, product2, date, address)
            i += 1
                
        order_id += 1
        orders_amount -=1
        
    month_name = calendar.month_name[month_value]
    print(month_name + ' Finished!!')
    df.to_excel(f"../data/processed/{month_name}_data.xlsx")
    break


January Finished!!


In [12]:
df.head()

,orderid,product,quantity,price,orderdate,address
0,14356,Palace Universal Case,1,129,2020-01-31 12:14:55.797447,"Alfredo Salazar 154, Miraflores, Lima"
1,14357,Huawei Y6 Smartphone,1,1149,2020-01-19 14:07:20.303505,"Granada 135, Pueblo Libre, Lima"
2,14357,Beats Studio3 Headphones,1,1499,2020-01-19 14:07:20.303505,"Granada 135, Pueblo Libre, Lima"
3,14358,JBL Go2 Speaker,1,129,2020-01-09 14:38:49.974557,"Av. Javier Prado Oeste 770, Magdalena, Magdale..."
4,14359,Motorola MotoG8 Smartphone,1,789,2020-01-17 16:03:08.456576,"Nicolás de Piérola 140, San Miguel, Lima"


In [13]:
from arcgis.geocoding import geocode
AGENT = 'seipy2019-intermediate-gds'
geolocator = Nominatim(user_agent=AGENT)

In [14]:
# test one location
location = geolocator.geocode('Av. Cesar Vallejo 223, Lince, Lince, Lima')
print(location)
location.raw
location.latitude

Avenida Cesar Vallejo, Lince, Lima, Lima Metropolitana, Lima, LIMA27, Perú


-12.0881541

In [15]:
# Obtain the latitude and longitude
addresses = df['address'].drop_duplicates().to_list()
df2 = pd.DataFrame({'address':addresses})

In [16]:
df2['latitude'] = df2['address'].apply(lambda x: None if pd.isnull(geolocator.geocode(x)) else geolocator.geocode(x).latitude)

In [17]:
df2['longitude'] = df2['address'].apply(lambda x: None if pd.isnull(geolocator.geocode(x)) else geolocator.geocode(x).longitude)


In [18]:
df2.to_excel('../data/processed/locations.xlsx', index=False)

In [19]:
locations_df = pd.read_excel('../data/processed/locations.xlsx')

In [20]:
df = pd.merge(df, locations_df, on='address', how='left')

In [21]:
# Remove null values in latitude ans longitude
df = df[~df['latitude'].isnull()].reset_index(drop=True)

In [22]:
import folium
from folium.plugins import HeatMap
m = folium.Map(location=[-12.087819, -77.045773], zoom_start=15)
tooltip = 'Click me'
folium.Marker([-12.087819, -77.045773], popup='<i> 2 transaccions</i>').add_to(m)

In [23]:
heat_data = [[row['latitude'], row['longitude']] for index, row in df.iterrows()]
HeatMap(heat_data).add_to(m)
m

In [24]:
locations_df

,address,latitude,longitude
0,"Alfredo Salazar 154, Miraflores, Lima",NaN,NaN
1,"Granada 135, Pueblo Libre, Lima",-12.068919,-77.064986
2,"Av. Javier Prado Oeste 770, Magdalena, Magdale...",-12.089805,-77.065908
3,"Nicolás de Piérola 140, San Miguel, Lima",-12.084542,-77.092367
4,"Av. 10 de Junio 1020, San Martín de Porres, Lima",-12.032744,-77.049490
5,"Av. Petit Thouars 1291, Santa Beatriz, Lima",-12.064312,-77.036324
6,"Micaela Bastidas 1100, El Retablo, Comas",-11.930953,-77.052290
7,"Av. Cuba 1145, Jesús María, Jesús María, Lima",-12.075069,-77.039725
8,"Av. Cesar Vallejo 223, Lince, Lince, Lima",-12.088154,-77.048063
9,"Av. Angamos Este 1551, Surquillo, Lima",-12.111435,-77.014594


In [ ]:
## Venta de Productos farmaceuticos online - Colombia
# Paciente crónicos : hipertención
# objetivo : la farmaceutica tiene como objetivo que siga luego de los 6 meses 
# seguros : predecir cuando el paciente va a dejar de consumir y evitar ir a la clinica.
# cruce en la prescripción de productos y evitar situaciones criticas
# clinicas necesitan clientes mas leales : 
# edad, peso, sexo, datos de su actividad fisica, nombre, beneficiario
# alergias
# tiroides, hipo, enfermedades 
# datos del pedido, vademecum y precio, datos del paciente: dirección, médico (condición crónica)
# https://www.datos.gov.co/en/widgets/p783-486i

# tricket promedio,